In [1]:
#script from homework of Coursera Neural Networks and Deep Learning
#get the input data from repo bayesian_logistic_regression

In [2]:
import numpy as np
import pandas as pd
import copy

In [3]:
df = pd.read_csv("../data/Bay_LR_Covid/covid_updated.csv")
X = df.iloc[:, :-1].values.T
y = df.iloc[:, -1:].values.T

In [4]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

In [5]:
def layer_sizes(X, Y):
    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]

    return (n_x, n_h, n_y)

In [6]:
def initialize_parameters(n_x, n_h, n_y):
    W1 = np.random.randn(n_h,n_x)* 0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)* 0.01
    b2 = np.zeros((n_y,1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [7]:
def forward_propagation(X, parameters):

    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    Z1 = np.dot(W1, X)+b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1)+b2
    A2 = sigmoid(Z2)

    assert(A2.shape == (1, X.shape[1]))

    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}

    return A2, cache

In [8]:
def compute_cost(A2, Y):

    m = Y.shape[1] # number of examples

    # Compute the cross-entropy cost
    logprobs = np.dot(Y,np.log(A2).T)+np.dot((1-Y),np.log(1-A2).T)
    cost = -logprobs/m
    cost = float(np.squeeze(cost))  # E.g., turns [[17]] into 17

    return cost

In [9]:
def backward_propagation(parameters, cache, X, Y):

    m = X.shape[1]

    W1 = parameters['W1']
    W2 = parameters['W2']

    A1 = cache['A1']
    A2 = cache['A2']

    dZ2 = A2-Y
    dW2 = np.dot(dZ2, A1.T)/m
    db2 = np.sum(dZ2,axis=1, keepdims=True)/m
    dZ1 = np.dot(W2.T, dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1,axis=1, keepdims=True)/m

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}

    return grads

In [10]:
def update_parameters(parameters, grads, learning_rate = 1.2):

    W1 = copy.deepcopy(parameters['W1'])
    b1 = parameters['b1']
    W2 = copy.deepcopy(parameters['W2'])
    b2 = parameters['b2']

    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']

    W1 = W1-learning_rate*dW1
    b1 = b1-learning_rate*db1
    W2 = W2-learning_rate*dW2
    b2 = b2-learning_rate*db2

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [11]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):

    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]

    # Initialize parameters
    parameters = initialize_parameters(n_x, n_h, n_y)

    #gradient descent
    for i in range(0, num_iterations):

        A2, cache = forward_propagation(X, parameters)
        cost = compute_cost(A2, Y)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grads)


        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [12]:
def predict(parameters, X):
    A2, cache = forward_propagation(X, parameters)

    predictions = np.where(A2>0.5, 1, 0)

    return predictions

In [13]:
parameters = nn_model(X, y, n_h = 4, num_iterations = 10000, print_cost=True)

Cost after iteration 0: 0.693030
Cost after iteration 1000: 0.146286
Cost after iteration 2000: 0.134018
Cost after iteration 3000: 0.129674
Cost after iteration 4000: 0.127463
Cost after iteration 5000: 0.125964
Cost after iteration 6000: 0.124853
Cost after iteration 7000: 0.123984
Cost after iteration 8000: 0.123280
Cost after iteration 9000: 0.122696


In [14]:
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(y, predictions.T) + np.dot(1 - y, 1 - predictions.T)) / float(y.size) * 100) + '%')

Accuracy: 95%
